In [1]:
from langchain_google_genai import ChatGoogleGenerativeAI
import os
model = ChatGoogleGenerativeAI(
                model='gemini-2.0-flash',
                google_api_key=os.getenv('GOOGLE_API_KEY'),
                temperature=0,
                max_output_tokens=1000
            )

model.invoke('Hello')

AIMessage(content='Hello! How can I help you today?', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.0-flash', 'safety_ratings': [], 'grounding_metadata': {}, 'model_provider': 'google_genai'}, id='lc_run--cf0922cd-79ea-4073-a444-f49edf1a2de7-0', usage_metadata={'input_tokens': 1, 'output_tokens': 10, 'total_tokens': 11, 'input_token_details': {'cache_read': 0}})

In [2]:
from langgraph.graph import START,END,StateGraph
from langgraph.checkpoint.memory import InMemorySaver
from typing import TypedDict

class JokeState(TypedDict):
    topic: str
    joke: str
    explanation: str


def generate_joke(state: JokeState):

    prompt = f'generate a joke on the topic {state["topic"]}'
    response = model.invoke(prompt).content

    return {'joke': response}

def generate_explanation(state: JokeState):

    prompt = f'write an explanation for the joke - {state["joke"]}'
    response = model.invoke(prompt).content

    return {'explanation': response}

graph = StateGraph(JokeState)
graph.add_node('generate_joke', generate_joke)
graph.add_node('generate_explanation', generate_explanation)
graph.add_edge(START, 'generate_joke')
graph.add_edge('generate_joke', 'generate_explanation')
graph.add_edge('generate_explanation', END)

checkpointer = InMemorySaver()

workflow = graph.compile(checkpointer=checkpointer)

In [3]:
config1 = {"configurable": {"thread_id": "1"}}
workflow.invoke({'topic':'pizza'}, config=config1)
workflow.get_state(config1)
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza maker quit his job?\n\nBecause he was tired of getting dough-pressed!', 'explanation': 'The joke "Why did the pizza maker quit his job? Because he was tired of getting dough-pressed!" relies on a pun, using the word "dough-pressed" in two different ways:\n\n* **Literal meaning:** In the context of making pizza, "dough-pressed" refers to the physical act of pressing and shaping the dough to form the pizza base.  A pizza maker does this repeatedly throughout their shift.\n\n* **Figurative meaning:** "Dough-pressed" sounds like "hard-pressed," which means feeling stressed, pressured, or overwhelmed.  The joke implies the pizza maker was feeling overworked and under pressure at their job.\n\nThe humor comes from the unexpected connection between the literal action of making pizza and the feeling of being stressed at work. The listener initially thinks about the physical act of pressing dough, but the punchline reveals the 

In [4]:
workflow.get_state({"configurable": {"thread_id": "1", "checkpoint_id": "1f0c15ff-2333-67ff-8001-86a997f81b71"}})

StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza maker quit his job?\n\nBecause he was tired of getting dough-pressed!'}, next=('generate_explanation',), config={'configurable': {'thread_id': '1', 'checkpoint_id': '1f0c15ff-2333-67ff-8001-86a997f81b71'}}, metadata={'source': 'loop', 'step': 1, 'parents': {}}, created_at='2025-11-14T13:43:47.798220+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0c15ff-1941-6e70-8000-905dbcf70323'}}, tasks=(PregelTask(id='c08c55f7-42d4-8190-cb20-94f089c9a8e3', name='generate_explanation', path=('__pregel_pull', 'generate_explanation'), error=None, interrupts=(), state=None, result={'explanation': 'The joke "Why did the pizza maker quit his job? Because he was tired of getting dough-pressed!" relies on a pun, using the word "dough-pressed" in two different ways:\n\n* **Literal meaning:** In the context of making pizza, "dough-pressed" refers to the physical act of pressing and shaping t

In [5]:
workflow.invoke(None, {"configurable": {"thread_id": "1", "checkpoint_id": "1f0c15ff-2333-67ff-8001-86a997f81b71"}})

{'topic': 'pizza',
 'joke': 'Why did the pizza maker quit his job?\n\nBecause he was tired of getting dough-pressed!',
 'explanation': 'The joke "Why did the pizza maker quit his job? Because he was tired of getting dough-pressed!" relies on a pun, using the word "dough-pressed" in two different ways:\n\n* **Literal meaning:** In the context of making pizza, "dough-pressed" refers to the physical act of pressing and shaping the dough to form the pizza base.  A pizza maker does this repeatedly throughout their shift.\n\n* **Figurative meaning:** "Dough-pressed" sounds like "hard-pressed," which means feeling stressed, pressured, or overwhelmed.  The joke implies the pizza maker was feeling overworked and under pressure at their job.\n\nThe humor comes from the unexpected connection between the literal action of making pizza and the feeling of being stressed at work. The listener initially thinks about the physical act of pressing dough, but the punchline reveals the double meaning, crea

In [13]:
workflow.update_state({"configurable": {"thread_id": "1", "checkpoint_id": "1f0c15ff-193e-6d11-bfff-8461124f2008", "checkpoint_ns": ""}}, {'topic':'samosa'})

{'configurable': {'thread_id': '1',
  'checkpoint_ns': '',
  'checkpoint_id': '1f0c1615-23df-6a68-8000-8db8a7d6a8ae'}}

In [14]:
list(workflow.get_state_history(config1))

[StateSnapshot(values={'topic': 'samosa'}, next=('generate_joke',), config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0c1615-23df-6a68-8000-8db8a7d6a8ae'}}, metadata={'source': 'update', 'step': 0, 'parents': {}}, created_at='2025-11-14T13:53:38.426736+00:00', parent_config={'configurable': {'thread_id': '1', 'checkpoint_ns': '', 'checkpoint_id': '1f0c15ff-193e-6d11-bfff-8461124f2008'}}, tasks=(PregelTask(id='7c45f5d1-db31-a22e-2665-fb7862ecb159', name='generate_joke', path=('__pregel_pull', 'generate_joke'), error=None, interrupts=(), state=None, result=None),), interrupts=()),
 StateSnapshot(values={'topic': 'pizza', 'joke': 'Why did the pizza maker quit his job?\n\nBecause he was tired of getting dough-pressed!', 'explanation': 'The joke "Why did the pizza maker quit his job? Because he was tired of getting dough-pressed!" is a pun that plays on the double meaning of the word "dough-pressed."\n\nHere\'s the breakdown:\n\n* **Literal meaning:** In th

In [15]:
workflow.invoke(None, {"configurable": {"thread_id": "1", "checkpoint_id": "1f0c1615-23df-6a68-8000-8db8a7d6a8ae"}})

{'topic': 'samosa',
 'joke': 'Why did the samosa break up with the chutney?\n\nBecause it felt like it was being used! It said, "I\'m tired of being dipped on!"',
 'explanation': 'The joke plays on the double meaning of the word "used" and the common practice of eating samosas with chutney. Here\'s a breakdown:\n\n* **Literal Meaning:** Samosas are often dipped in chutney to enhance their flavor. This is the intended, normal way to consume them.\n* **Figurative Meaning:** The samosa feels "used" in a relationship sense. It feels like it\'s only valued for its utility (being dipped in chutney) and not for its own sake.\n* **"Dipped on"** is a pun. It refers to:\n    * **The literal act of dipping:** The samosa is constantly being dipped in chutney.\n    * **Being taken advantage of:** "Dipped on" can also mean being taken advantage of or treated poorly in a relationship.\n\nTherefore, the joke is funny because it anthropomorphizes the samosa, giving it human-like feelings of being used 

In [2]:
from langgraph.graph import StateGraph, END
from langgraph.checkpoint.memory import InMemorySaver
from typing import TypedDict
import time

class CrashState(TypedDict):
    input: str
    step1: str
    step2: str

def step_1(state: CrashState) -> CrashState:
    print("✅ Step 1 executed")
    return {"step1": "done", "input": state["input"]}

def step_2(state: CrashState) -> CrashState:
    print("⏳ Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)")
    time.sleep(1000)  # Simulate long-running hang
    return {"step2": "done"}

def step_3(state: CrashState) -> CrashState:
    print("✅ Step 3 executed")
    return {"done": True}

builder = StateGraph(CrashState)
builder.add_node("step_1", step_1)
builder.add_node("step_2", step_2)
builder.add_node("step_3", step_3)

builder.set_entry_point("step_1")
builder.add_edge("step_1", "step_2")
builder.add_edge("step_2", "step_3")
builder.add_edge("step_3", END)

checkpointer = InMemorySaver()
graph = builder.compile(checkpointer=checkpointer)

In [ ]:
try:
    print("▶️ Running graph: Please manually interrupt during Step 2...")
    graph.invoke({"input": "start"}, config={"configurable": {"thread_id": 'thread-1'}})
except KeyboardInterrupt:
    print("❌ Kernel manually interrupted (crash simulated).")

▶️ Running graph: Please manually interrupt during Step 2...
✅ Step 1 executed
⏳ Step 2 hanging... now manually interrupt from the notebook toolbar (STOP button)
